In [97]:
import pandas as pd
from IPython.display import Markdown
from IPython.core.debugger import set_trace as breakpt

In [92]:
# Read in the seat input
seatids = list()
with open('seats_input.txt', 'r') as fid:
    for line in fid:
        seatids.append(line.strip())

In [17]:
c1 = 'FBFFFFBRRL'
s1 = list(range(8))

In [36]:
def binpart(code, segment):
    mid = len(segment) // 2
    if code in 'FL':
        return segment[:mid]
    elif code in 'BR':
        return segment[mid:]
    else:
        raise ValueError("Unhandled code: {}".format(code))

In [40]:
print(binpart('F', s1))
print(binpart('B', s1))
print(binpart('L', s1))
print(binpart('R', s1))

[0, 1, 2, 3]
[4, 5, 6, 7]
[0, 1, 2, 3]
[4, 5, 6, 7]


In [151]:
def consume(codestr, segment, result=None):
    #print("codestr: {}, segment: {}, len: {}, result: {}"
    #      .format(codestr, segment, len(segment), result))
    if 1 == len(segment):
        print("HIT")
        return result
    else:
        seg = binpart(codestr[0], segment)
        result = seg
        #print("codestr: {}, code: {}, seg: {}, result: {}"
        #      .format(codestr, codestr[0], seg, result))
        return consume(codestr[1:], seg, result)

a = consume(c1, s1)
print(a)        

codestr: FBFFFFBRRL, segment: [0, 1, 2, 3, 4, 5, 6, 7], len: 8, result: None
codestr: BFFFFBRRL, segment: [0, 1, 2, 3], len: 4, result: [0, 1, 2, 3]
codestr: FFFFBRRL, segment: [2, 3], len: 2, result: [2, 3]
codestr: FFFBRRL, segment: [2], len: 1, result: [2]
HIT
[2]


In [152]:
a = consume(c1, s1)
print(a)

codestr: FBFFFFBRRL, segment: [0, 1, 2, 3, 4, 5, 6, 7], len: 8, result: None
codestr: BFFFFBRRL, segment: [0, 1, 2, 3], len: 4, result: [0, 1, 2, 3]
codestr: FFFFBRRL, segment: [2, 3], len: 2, result: [2, 3]
codestr: FFFBRRL, segment: [2], len: 1, result: [2]
HIT
[2]


In [72]:
s = s1
for codechar in 'FBF':
    s = binpart(codechar, s)
s

[2]

In [80]:
def getbin(codestr, partsize):
    seg = list(range(partsize))
    for codechar in codestr:
        seg = binpart(codechar, seg)
    return seg[0]

#getbin(c1[:7], 128)
#getbin(c1[7:], 8)

In [93]:
seatpos = list()
rows = list()
cols = list()
for i in seatids:
    r = getbin(i[:7], 128)
    c = getbin(i[7:], 8)
    seatpos.append((r, c))
    rows.append(r)
    cols.append(c)

In [94]:
ids = [r * 8 + c for r, c in seatpos]

In [95]:
max(ids)

822

In [112]:
df = pd.DataFrame({'row': rows, 'col': cols, 'seatid': ids})
df

,row,col,seatid
0,87,4,700
1,79,7,639
2,26,0,208
3,31,5,253
4,74,6,598
...,...,...,...
804,35,3,283
805,60,5,485
806,52,5,421
807,44,6,358


In [113]:
df1 = df.sort_values(['row', 'col'])
df1

,row,col,seatid
739,1,5,13
204,1,6,14
10,1,7,15
689,2,0,16
562,2,1,17
...,...,...,...
596,102,2,818
514,102,3,819
203,102,4,820
427,102,5,821


In [117]:
rowmin = min(rows)
rowmax = max(rows)

print(rowmin, rowmax)

1 102


In [133]:
df2 = df1.sort_values('seatid')
df2['delta'] = df2['seatid'] - df2['seatid'].shift(1)
df2 = df2.reset_index()
df2

,index,row,col,seatid,delta
0,739,1,5,13,NaN
1,204,1,6,14,1.0
2,10,1,7,15,1.0
3,689,2,0,16,1.0
4,562,2,1,17,1.0
...,...,...,...,...,...
804,596,102,2,818,1.0
805,514,102,3,819,1.0
806,203,102,4,820,1.0
807,427,102,5,821,1.0


In [134]:
df2[df2['delta'] != 1]

,index,row,col,seatid,delta
0,739,1,5,13,NaN
692,487,88,2,706,2.0


In [137]:
df2.loc[690:695]

,index,row,col,seatid,delta
690,530,87,7,703,1.0
691,228,88,0,704,1.0
692,487,88,2,706,2.0
693,784,88,3,707,1.0
694,324,88,4,708,1.0
695,210,88,5,709,1.0
